In [1]:
import pathlib
import sqlite3
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import black
import jupyter_black

jupyter_black.load(
    lab=False,
    line_length=79,
    verbosity="DEBUG",
    target_version=black.TargetVersion.PY310,
)

# ignore mix type warnings from pandas
import warnings

warnings.filterwarnings("ignore")

DEBUG:jupyter_black:config: {'line_length': 79, 'target_versions': {<TargetVersion.PY310: 10>}}


<IPython.core.display.Javascript object>

In [2]:
# Setting file paths
data_dir = pathlib.Path("../../").resolve(strict=True)
metadata_dir = pathlib.Path(
    "../../2021_09_01_VarChAMP-data/metadata/reprocessed/"
).resolve(strict=True)
out_dir = pathlib.Path("results")
out_dir.mkdir(exist_ok=True)

# input file paths
plate = "2023-05-30_B1A1R1_P1T1"
plate_data = pathlib.Path(data_dir / "2023-05-30_B1A1R1_P1T1.sqlite").resolve(
    strict=True
)
plate_map = (metadata_dir / "2023_05_30_B1A1R1.csv").resolve(strict=True)

# setting output paths
sc_profiles_path = out_dir / "B1A1R1_P1T1_single_cell_profile.csv.gz"

In [3]:
def is_feature_col(col):
    """Check if column is a feature"""
    return (
        col.startswith("Cell")
        or col.startswith("Cytoplasm")
        or col.startswith("Nuclei")
    )


def count(cur, table):
    """Count total number of rows for a table"""
    (num_rows,) = next(cur.execute(f"SELECT COUNT(*) FROM {table}"))
    return num_rows


def get_columns(cur, table):
    """Get feature and metadata columns lists"""
    ptr = cur.execute(f"SELECT * FROM {table} LIMIT 1")
    col_names = [obj[0] for obj in ptr.description]

    feat_cols = []
    meta_cols = []
    for col in col_names:
        if is_feature_col(col):
            feat_cols.append(col)
        else:
            meta_cols.append(col)

    return meta_cols, feat_cols


def load_compartment(cur, table):
    # Get data useful to pre-alloc memory
    num_cells = count(cur, table)
    meta_cols, feat_cols = get_columns(cur, table)
    num_meta, num_feats = len(meta_cols), len(feat_cols)

    feats = np.empty(shape=(num_cells, num_feats), dtype=np.float32)
    meta = pd.DataFrame(columns=meta_cols, index=range(num_cells))

    columns = ", ".join(meta_cols + feat_cols)
    query = f"SELECT {columns} from {table}"
    resultset = cur.execute(query)
    for i, row in tqdm(enumerate(resultset), total=num_cells):
        meta.loc[i] = row[:num_meta]
        feats[i] = row[num_meta:]
    return pd.concat(
        [meta, pd.DataFrame(columns=feat_cols, data=feats)], axis=1
    )

In [4]:
# update compartment names and strata
strata = ["Metadata_Well", "Metadata_Plate"]
compartments = ["Cells", "Nuclei", "Cytoplasm"]

# Updating linking columns for merging all compartments
linking_cols = {
    "Cytoplasm": {
        "Cells": "Cytoplasm_Parent_Cells",
        "Nuclei": "Cytoplasm_Parent_Nuclei",
    },
    "Cells": {"Cytoplasm": "Cells_Number_Object_Number"},
    "Nuclei": {"Cytoplasm": "Nuclei_Number_Object_Number"},
}

merge_cols = ["ImageNumber", "TableNumber"]

In [5]:
con = sqlite3.connect(plate_data)
cur = con.cursor()

In [6]:
sc_df = ""
linking_check_cols = []
merge_suffix_rename = []

for left_compartment in linking_cols:
    for right_compartment in linking_cols[left_compartment]:
        # Make sure only one merge per combination occurs
        linking_check = "-".join(sorted([left_compartment, right_compartment]))
        if linking_check in linking_check_cols:
            continue

        # Specify how to indicate merge suffixes
        merge_suffix = [
            "_{comp_l}".format(comp_l=left_compartment),
            "_{comp_r}".format(comp_r=right_compartment),
        ]
        merge_suffix_rename += merge_suffix
        left_link_col = linking_cols[left_compartment][right_compartment]
        right_link_col = linking_cols[right_compartment][left_compartment]

        if isinstance(sc_df, str):
            sc_df = load_compartment(cur, left_compartment)

        sc_df = sc_df.merge(
            load_compartment(cur, right_compartment),
            left_on=merge_cols + [left_link_col],
            right_on=merge_cols + [right_link_col],
            suffixes=merge_suffix,
        )

        linking_check_cols.append(linking_check)

  0%|          | 0/168383 [00:00<?, ?it/s]

  0%|          | 0/168383 [00:00<?, ?it/s]

  0%|          | 0/168383 [00:00<?, ?it/s]

In [7]:
rowdict = {
    "01": "A",
    "02": "B",
    "03": "C",
    "04": "D",
    "05": "E",
    "06": "F",
    "07": "G",
    "08": "H",
    "09": "I",
    "10": "J",
    "11": "K",
    "12": "L",
    "13": "M",
    "14": "N",
    "15": "O",
    "16": "P",
}

image_df = load_compartment(cur, "Image")
image_df = image_df[["TableNumber", "FileName_OrigAGP"]]
image_df["Metadata_Plate"] = plate
image_df["Metadata_Well"] = image_df["FileName_OrigAGP"].apply(
    lambda x: rowdict[x[1:3]] + x[4:6]
)
aligned_df = sc_df.merge(image_df, on=["TableNumber"], how="left")
aligned_df.drop(
    [
        "FileName_OrigAGP",
        "Cytoplasm_Parent_Cells",
        "Cytoplasm_Parent_Nuclei",
        "Cells_Number_Object_Number",
        "Nuclei_Number_Object_Number",
        "ObjectNumber_Cytoplasm",
        "ObjectNumber",
        "ObjectNumber_Cells",
        "TableNumber",
        "ImageNumber",
    ],
    inplace=True,
    axis=1,
)

meta_df = pd.read_csv(plate_map)
meta_df = meta_df[
    [
        "Metadata_Plate",
        "Metadata_Well",
        "node_type",
        "allele",
        "batch",
        "control",
        "Gene",
        "MT",
        "Variant",
        "Metadata_Sample_Unique",
        "Metadata_batch_Plate",
    ]
]

col_name = []
for i in meta_df.columns:
    if "Metadata" in i:
        col_name.append(i)
    else:
        col_name.append("Metadata_" + i)

meta_df = meta_df.set_axis(col_name, axis=1)

aligned_df = meta_df.merge(
    aligned_df, on=["Metadata_Plate", "Metadata_Well"], how="right"
)
aligned_df

  0%|          | 0/3370 [00:00<?, ?it/s]

,Metadata_Plate,Metadata_Well,Metadata_node_type,Metadata_allele,Metadata_batch,Metadata_control,Metadata_Gene,Metadata_MT,Metadata_Variant,Metadata_Sample_Unique,...,Nuclei_Texture_Variance_Mito_10_02_256,Nuclei_Texture_Variance_Mito_10_03_256,Nuclei_Texture_Variance_Mito_20_00_256,Nuclei_Texture_Variance_Mito_20_01_256,Nuclei_Texture_Variance_Mito_20_02_256,Nuclei_Texture_Variance_Mito_20_03_256,Nuclei_Texture_Variance_Mito_5_00_256,Nuclei_Texture_Variance_Mito_5_01_256,Nuclei_Texture_Variance_Mito_5_02_256,Nuclei_Texture_Variance_Mito_5_03_256
0,2023-05-30_B1A1R1_P1T1,A01,disease_wt,ACSF3_,2023_05_30_B1A1R1,False,ACSF3,NaN,ACSF3,ACSF3,...,0.103070,0.104598,0.099677,0.000000,0.074942,0.117092,0.112644,0.110582,0.109753,0.104428
1,2023-05-30_B1A1R1_P1T1,A01,disease_wt,ACSF3_,2023_05_30_B1A1R1,False,ACSF3,NaN,ACSF3,ACSF3,...,0.938335,0.813760,0.000000,0.000000,0.000000,0.000000,0.908367,0.871419,0.906835,0.930216
2,2023-05-30_B1A1R1_P1T1,A01,disease_wt,ACSF3_,2023_05_30_B1A1R1,False,ACSF3,NaN,ACSF3,ACSF3,...,1.049312,1.382822,0.612253,0.326389,1.266040,0.782384,1.038961,1.079116,0.959314,1.058321
3,2023-05-30_B1A1R1_P1T1,A01,disease_wt,ACSF3_,2023_05_30_B1A1R1,False,ACSF3,NaN,ACSF3,ACSF3,...,3.673092,3.672637,3.478531,7.463806,5.229506,2.418224,3.365278,3.451281,3.233617,3.455639
4,2023-05-30_B1A1R1_P1T1,A01,disease_wt,ACSF3_,2023_05_30_B1A1R1,False,ACSF3,NaN,ACSF3,ACSF3,...,8.363085,10.306902,11.090281,7.700174,11.944907,26.675154,7.730883,8.747192,7.624651,8.027596
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168377,2023-05-30_B1A1R1_P1T1,P24,allele,FBP1_Ala177Asp,2023_05_30_B1A1R1,False,FBP1,Ala177Asp,FBP1 Ala177Asp,FBP1 Ala177Asp,...,9.105786,8.171266,0.000000,0.000000,0.000000,0.000000,8.073656,9.197808,8.734521,8.206161
168378,2023-05-30_B1A1R1_P1T1,P24,allele,FBP1_Ala177Asp,2023_05_30_B1A1R1,False,FBP1,Ala177Asp,FBP1 Ala177Asp,FBP1 Ala177Asp,...,4.325852,6.402656,0.000000,0.000000,0.000000,0.000000,6.235636,5.166418,5.147982,6.113783
168379,2023-05-30_B1A1R1_P1T1,P24,allele,FBP1_Ala177Asp,2023_05_30_B1A1R1,False,FBP1,Ala177Asp,FBP1 Ala177Asp,FBP1 Ala177Asp,...,7.027832,1.270007,6.217703,13.561917,7.857816,6.275992,5.056368,5.904297,5.984637,3.287448
168380,2023-05-30_B1A1R1_P1T1,P24,allele,FBP1_Ala177Asp,2023_05_30_B1A1R1,False,FBP1,Ala177Asp,FBP1 Ala177Asp,FBP1 Ala177Asp,...,298.326324,386.666504,272.396088,171.617279,347.785828,377.873260,309.126373,304.271973,309.399445,319.596100


In [9]:
aligned_df.to_csv(sc_profiles_path, index=False, compression="gzip")

In [45]:
# sql_query = """SELECT name FROM sqlite_master  
#   WHERE type='table';"""
# (num_rows,) = next(cur.execute('''SELECT COUNT(*) FROM Image'''))